In [ ]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import anndata as ad
# import numpy as np
# import pandas as pd

# input_dir = "/Users/apple/Desktop/KB/data"
# # Load dataset

# X_train = np.load(input_dir+'/feat_train_test/biddy/bs30_sf0025/scBaseEncoderFeat_Z_bs30_tau0.5.npy')
# print(X_train.shape)

# adata_train = ad.read_h5ad(input_dir+'/BiddyData/Biddy_train.h5ad')

# # Replace NaN values in the 'cell_type' column with "undifferentiated"
# adata_train.obs["cell_type"] = adata_train.obs["cell_type"].cat.add_categories("undifferentiated").fillna("undifferentiated")

# # Verify the replacement
# print(adata_train.obs["cell_type"].unique())

# # adata_train.obs['Cell type annotation'] = adata_train.obs["cell_type"]
# # adata_test.obs['Cell type annotation'] = adata_test.obs["cell_type"]
# train_labels = adata_train.obs["clone_id"].to_numpy()
# print(train_labels.shape)

# adata_28 = adata_train[adata_train.obs["reprogramming_day"] == "28"]
# print(adata_28.shape)

# # Initialize an empty dictionary to store the cell type distributions
# clone_cell_type_distribution = {}

# # Get the unique clone IDs
# unique_clone_ids = adata_28.obs["clone_id"].unique()

# # Loop through each unique clone_id
# for clone_id in unique_clone_ids:
#     # Filter the data to get only rows with the current clone_id
#     clone_data = adata_28.obs[adata_28.obs["clone_id"] == clone_id]
    
#     # Get the distribution of cell types in the current clone_id
#     cell_type_distribution = clone_data["cell_type"].value_counts(normalize=True)
    
#     # Round each percentage to 4 decimal places and convert to a dictionary
#     cell_type_distribution = cell_type_distribution.round(4).to_dict()
    
#     # Store the rounded distribution in the main dictionary
#     clone_cell_type_distribution[clone_id] = cell_type_distribution


# # Step 1: Get embeddings for Day 12 cells
# day12_mask = adata_train.obs["reprogramming_day"] == "12"  # Assuming '12' represents Day 12
# X_train_day12 = X_train[day12_mask.values]  # Get embeddings corresponding to Day 12
# print(f"Day 12 embeddings shape: {X_train_day12.shape}")

# # Step 2: Get the clone labels for Day 12 cells
# clone_labels_day12 = adata_train.obs.loc[day12_mask, "clone_id"].to_numpy()

# # Step 3: Initialize y_train_prob matrix to store the probabilities
# # There are now 4 possible cell types: ['iEP', 'undifferentiated', 'Ambiguous', 'Fibroblast']
# n_classes = 4
# y_train_prob = np.zeros((X_train_day12.shape[0], n_classes))

# # Step 4: Assign the distributions from clone_cell_type_distribution to each cell based on its clone_id
# for i, clone_id in enumerate(clone_labels_day12):
#     if clone_id in clone_cell_type_distribution:
#         # Get the distribution for the clone
#         distribution = clone_cell_type_distribution[clone_id]
        
#         # Ensure the order of cell types matches ['iEP', 'undifferentiated', 'Ambiguous', 'Fibroblast']
#         y_train_prob[i, 0] = distribution.get('iEP', 0)  # Default to 0 if not present
#         y_train_prob[i, 1] = distribution.get('undifferentiated', 0)  # Default to 0 if not present
#         y_train_prob[i, 2] = distribution.get('Ambiguous', 0)  # Default to 0 if not present
#         y_train_prob[i, 3] = distribution.get('Fibroblast', 0)  # Default to 0 if not present

# # Print the shape and first few examples of y_train_prob
# print(f"y_train_prob shape: {y_train_prob.shape}")
# print(f"First 5 rows of y_train_prob:\n{y_train_prob[:5]}")



# X_train_day12 = torch.tensor(X_train_day12, dtype=torch.float32)

# # Example soft labels: 5 samples, each with a probability distribution over 3 classes
# y_train_prob = torch.tensor(y_train_prob, dtype=torch.float32)


# # Define a simple neural network model
# class SoftLabelNN(nn.Module):
#     def __init__(self, input_size, hidden_size, output_size):
#         super(SoftLabelNN, self).__init__()
#         self.fc1 = nn.Linear(input_size, hidden_size)
#         self.relu = nn.ReLU()
#         self.fc2 = nn.Linear(hidden_size, output_size)
#         self.softmax = nn.Softmax(dim=1)
    
#     def forward(self, x):
#         out = self.fc1(x)
#         out = self.relu(out)
#         out = self.fc2(out)
#         out = self.softmax(out)
#         return out
    

# # Initialize the model, loss function, and optimizer
# input_size = X_train_day12.shape[1]
# hidden_size = 10
# output_size = y_train_prob.shape[1]

# model = SoftLabelNN(input_size, hidden_size, output_size)
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.01)

# # Convert soft labels to one-hot encoding for the CrossEntropyLoss
# # (PyTorch expects class indices for CrossEntropyLoss, not probabilities)
# y_train_indices = torch.argmax(y_train_prob, dim=1)

# # Training loop
# num_epochs = 10000000
# for epoch in range(num_epochs):
#     # Forward pass
#     outputs = model(X_train_day12)
#     loss = criterion(outputs, y_train_indices)
    
#     # Backward pass and optimization
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()
    
#     if (epoch+1) % 50 == 0:
#         print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')




In [1]:

import torch
import torch.nn as nn
import torch.optim as optim
import anndata as ad
import numpy as np
import pandas as pd


In [2]:
input_dir = "/Users/apple/Desktop/KB/data"
# Load dataset

X_train = np.load(input_dir+'/feat_train_test/biddy/bs30_sf0025/scBaseEncoderFeat_Z_bs30_tau0.5.npy')
print(X_train.shape)

adata_train = ad.read_h5ad(input_dir+'/BiddyData/Biddy_train.h5ad')

# Replace NaN values in the 'cell_type' column with "undifferentiated"
adata_train.obs["cell_type"] = adata_train.obs["cell_type"].cat.add_categories("undifferentiated").fillna("undifferentiated")

# Verify the replacement
print(adata_train.obs["cell_type"].unique())

# adata_train.obs['Cell type annotation'] = adata_train.obs["cell_type"]
# adata_test.obs['Cell type annotation'] = adata_test.obs["cell_type"]
train_labels = adata_train.obs["clone_id"].to_numpy()
print(train_labels.shape)


(9220, 64)
['iEP', 'undifferentiated', 'Ambiguous', 'Fibroblast']
Categories (4, object): ['Ambiguous', 'Fibroblast', 'iEP', 'undifferentiated']
(9220,)


#### Generate a dict to store the cell type distribution for each lineage 

In [3]:
adata_28 = adata_train[adata_train.obs["reprogramming_day"] == "28"]
print(adata_28.shape)

# Initialize an empty dictionary to store the cell type distributions
clone_cell_type_distribution = {}

# Get the unique clone IDs
unique_clone_ids = adata_28.obs["clone_id"].unique()

# Loop through each unique clone_id
for clone_id in unique_clone_ids:
    # Filter the data to get only rows with the current clone_id
    clone_data = adata_28.obs[adata_28.obs["clone_id"] == clone_id]
    
    # Get the distribution of cell types in the current clone_id
    cell_type_distribution = clone_data["cell_type"].value_counts(normalize=True)
    
    # Round each percentage to 4 decimal places and convert to a dictionary
    cell_type_distribution = cell_type_distribution.round(4).to_dict()
    
    # Store the rounded distribution in the main dictionary
    clone_cell_type_distribution[clone_id] = cell_type_distribution

(3239, 2000)


In [4]:
# Print the resulting dictionary for verification
i = 0
for clone_id, distribution in clone_cell_type_distribution.items():
    print(f"Clone ID: {clone_id}, Cell Type Distribution: {distribution}")
    i+=1
    if i ==5:
        break

Clone ID: 493.0, Cell Type Distribution: {'iEP': 0.6573, 'undifferentiated': 0.2571, 'Ambiguous': 0.0631, 'Fibroblast': 0.0226}
Clone ID: 2352.0, Cell Type Distribution: {'Fibroblast': 0.3806, 'undifferentiated': 0.2985, 'Ambiguous': 0.2463, 'iEP': 0.0746}
Clone ID: 487.0, Cell Type Distribution: {'iEP': 0.4872, 'undifferentiated': 0.2821, 'Ambiguous': 0.2179, 'Fibroblast': 0.0128}
Clone ID: 666.0, Cell Type Distribution: {'iEP': 0.494, 'undifferentiated': 0.4194, 'Fibroblast': 0.0524, 'Ambiguous': 0.0343}
Clone ID: 2721.0, Cell Type Distribution: {'iEP': 0.4103, 'Ambiguous': 0.2051, 'undifferentiated': 0.2051, 'Fibroblast': 0.1795}


#### Get the scCL embeddings for Day 12 cell and assign the cell type distribution label for it 

In [5]:
# Step 1: Get embeddings for Day 12 cells
day12_mask = adata_train.obs["reprogramming_day"] == "12"  # Assuming '12' represents Day 12
X_train_day12 = X_train[day12_mask.values]  # Get embeddings corresponding to Day 12
print(f"Day 12 embeddings shape: {X_train_day12.shape}")

# Step 2: Get the clone labels for Day 12 cells
clone_labels_day12 = adata_train.obs.loc[day12_mask, "clone_id"].to_numpy()

# Step 3: Initialize y_train_prob matrix to store the probabilities
# There are now 4 possible cell types: ['iEP', 'undifferentiated', 'Ambiguous', 'Fibroblast']
n_classes = 4
y_train_prob = np.zeros((X_train_day12.shape[0], n_classes))

# Step 4: Assign the distributions from clone_cell_type_distribution to each cell based on its clone_id
for i, clone_id in enumerate(clone_labels_day12):
    if clone_id in clone_cell_type_distribution:
        # Get the distribution for the clone
        distribution = clone_cell_type_distribution[clone_id]
        
        # Ensure the order of cell types matches ['iEP', 'undifferentiated', 'Ambiguous', 'Fibroblast']
        y_train_prob[i, 0] = distribution.get('iEP', 0)  # Default to 0 if not present
        y_train_prob[i, 1] = distribution.get('undifferentiated', 0)  # Default to 0 if not present
        y_train_prob[i, 2] = distribution.get('Ambiguous', 0)  # Default to 0 if not present
        y_train_prob[i, 3] = distribution.get('Fibroblast', 0)  # Default to 0 if not present

# Print the shape and first few examples of y_train_prob
print(f"y_train_prob shape: {y_train_prob.shape}")
print(f"First 5 rows of y_train_prob:\n{y_train_prob[:5]}")


Day 12 embeddings shape: (1477, 64)
y_train_prob shape: (1477, 4)
First 5 rows of y_train_prob:
[[0.6573 0.2571 0.0631 0.0226]
 [0.6573 0.2571 0.0631 0.0226]
 [0.6573 0.2571 0.0631 0.0226]
 [0.6573 0.2571 0.0631 0.0226]
 [0.6573 0.2571 0.0631 0.0226]]


#### Multi-label classifier with KL Loss

In [6]:
X_train_day12 = torch.tensor(X_train_day12, dtype=torch.float32)

# Example soft labels: 5 samples, each with a probability distribution over 3 classes
y_train_prob = torch.tensor(y_train_prob, dtype=torch.float32)
X_train_day12, y_train_prob

(tensor([[0.0000, 0.0000, 0.0000,  ..., 1.7789, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 1.7411, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 1.7587, 0.0000, 0.0000],
         ...,
         [1.7374, 0.0000, 0.0000,  ..., 0.0000, 0.1937, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]),
 tensor([[0.6573, 0.2571, 0.0631, 0.0226],
         [0.6573, 0.2571, 0.0631, 0.0226],
         [0.6573, 0.2571, 0.0631, 0.0226],
         ...,
         [0.0000, 0.0000, 0.0000, 0.0000],
         [1.0000, 0.0000, 0.0000, 0.0000],
         [1.0000, 0.0000, 0.0000, 0.0000]]))

In [7]:

# Define a simple neural network model
class SoftLabelNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SoftLabelNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)  # Raw output before softmax
        return out

# Initialize the model, optimizer, and KLDivLoss function
input_size = X_train_day12.shape[1]
hidden_size = 10
output_size = y_train_prob.shape[1]

model = SoftLabelNN(input_size, hidden_size, output_size)
criterion = nn.KLDivLoss(reduction='batchmean')  # KLDivLoss for comparing distributions
optimizer = optim.Adam(model.parameters(), lr=0.01)




In [8]:
# Training loop
num_epochs = 10000
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train_day12)
    
    # Apply log_softmax to get log probabilities
    outputs_log_prob = torch.log_softmax(outputs, dim=1)
    
    # Calculate the KL divergence loss between predicted and true distributions
    loss = criterion(outputs_log_prob, y_train_prob)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 50 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [50/10000], Loss: 0.0634
Epoch [100/10000], Loss: 0.0213
Epoch [150/10000], Loss: 0.0126
Epoch [200/10000], Loss: 0.0093
Epoch [250/10000], Loss: 0.0072
Epoch [300/10000], Loss: 0.0061
Epoch [350/10000], Loss: 0.0047
Epoch [400/10000], Loss: 0.0030
Epoch [450/10000], Loss: 0.0026
Epoch [500/10000], Loss: 0.0022
Epoch [550/10000], Loss: 0.0020
Epoch [600/10000], Loss: 0.0018
Epoch [650/10000], Loss: 0.0017
Epoch [700/10000], Loss: 0.0016
Epoch [750/10000], Loss: 0.0015
Epoch [800/10000], Loss: 0.0015
Epoch [850/10000], Loss: 0.0014
Epoch [900/10000], Loss: 0.0013
Epoch [950/10000], Loss: 0.0012
Epoch [1000/10000], Loss: 0.0012
Epoch [1050/10000], Loss: 0.0012
Epoch [1100/10000], Loss: 0.0012
Epoch [1150/10000], Loss: 0.0011
Epoch [1200/10000], Loss: 0.0011
Epoch [1250/10000], Loss: 0.0011
Epoch [1300/10000], Loss: 0.0010
Epoch [1350/10000], Loss: 0.0010
Epoch [1400/10000], Loss: 0.0010
Epoch [1450/10000], Loss: 0.0010
Epoch [1500/10000], Loss: 0.0009
Epoch [1550/10000], Loss: 0.00

In [ ]:
# Define a simple neural network model
class SoftLabelNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SoftLabelNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.softmax(out)
        return out

In [ ]:

# Initialize the model, loss function, and optimizer
input_size = X_train_day12.shape[1]
hidden_size = 10
output_size = y_train_prob.shape[1]

model = SoftLabelNN(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Convert soft labels to one-hot encoding for the CrossEntropyLoss
# (PyTorch expects class indices for CrossEntropyLoss, not probabilities)
y_train_indices = torch.argmax(y_train_prob, dim=1)

# Training loop
num_epochs = 10000000
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train_day12)
    loss = criterion(outputs, y_train_indices)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 50 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')




In [ ]:
# Example test data
X_test = torch.tensor([
    [0.2, 0.1],
    [0.3, 0.6]
], dtype=torch.float32)

# Predict the probability distribution over classes for new samples
with torch.no_grad():
    predicted_probabilities = model(X_test)
    print("Predicted Probabilities:")
    print(predicted_probabilities)